In [179]:
import requests
import csv as csv
import pandas as pd
import os 
import numpy as np
import psycopg2

In [126]:
# set the current conference to work with, e.g. HICSS-56, HICSS-55, HICSS-54
CONFERENCE = 'HICSS-55'

### Minitrack Proposal Downloads

In [2]:
import glob
import subprocess

# reference: https://stackoverflow.com/questions/60632421/is-there-any-package-for-converting-doc-file-to-docx-format-using-python-program
def convertDoc2Docx(dirname: str, filename: str):
	"""
	Utility functino to take in a .doc file and convert it 
	to .docx using libreoffice (soffice). This probably won't
	work in a Colab environment. 
	The filepath to soffice would need to be adjusted per OS.

	Args:
		dirname: str, directory where file is located
		filename: str, filename of the .doc file
	"""
	if filename.endswith('.doc'):
		curr_dir = os.getcwd()
		os.chdir(dirname)
		subprocess.call(['/Applications/LibreOffice.app/Contents/MacOS/soffice', '--headless', '--convert-to', 'docx', filename])
		os.chdir(curr_dir)

The following block of code iterates over csv files for each track, and then downloads the minitrack proposal for each accepted minitrack. The ES track doesn't have any proposals and is thus excluded. Since ES only has 4 minitracks, those can be manually inserted into the DB.

In [151]:
def formatMinitrackName(name: str):
	"""
	Used for formatting of filenames to avoid capitalization, special character 
	and spacing issues
	"""
	return name.replace('/', '^').replace(' ', '').lower()

In [154]:
#tracks = ['CL', 'DA', 'DSM', 'DG', 'ES', 'HC', 'IN', 'KS', 'LI', 'OS', 'ST']
tracks = ['CL', 'DA', 'DSM', 'DG', 'HC', 'IN', 'KS', 'LI', 'OS', 'ST']
# tracks = ['ST']
os.chdir('/Users/holmsmidt/hicss/workspace/track-recommender/minitrack-lists')
print(os.getcwd())
for track in tracks:
	minitracks = pd.read_csv(f"./{CONFERENCE}/proposals/{CONFERENCE} Minitrack Proposals - {track}.csv")
	# reference: https://appdividend.com/2021/07/03/how-to-create-directory-if-not-exist-in-python/
	path = f"./{CONFERENCE}/files/{minitracks['Track'][0]}"
	isExist = os.path.exists(path)
	if not isExist:
		os.makedirs(path)
	for index, row in minitracks.iterrows():
		if row['Decision (A/AM/R)'] == 'A':
			url = row['Proposal']
			# reference: https://www.tutorialspoint.com/downloading-files-from-web-using-python
			r = requests.get(url, stream=True)
			split = row['Proposal'].split('.')
			ext = split[len(split)-1]
			# reference: https://stackoverflow.com/questions/37913921/how-to-download-file-from-web-using-request-module
			# quick hack to deal with '/' characters in minitrack names by replacing with an unlikely character '^'
			filename = f"{formatMinitrackName(row['Proposed Minitrack'])}.{ext}"
			outfile = open(f"{path}/{filename}", 'wb')
			outfile.write(r.content)
			outfile.close()
			if ext == 'doc':
				# create .docx from .doc
				convertDoc2Docx(path, filename)
				# delete .doc
				os.remove(f"{path}/{filename}")
			

/Users/holmsmidt/hicss/workspace/track-recommender/minitrack-lists


### Extract Minitrack Descriptions from Files

In [130]:
tracksdict = {
	'CL': 'Collaboration Systems and Technologies',
	'DA': 'Decision Analytics and Service Science', 
	'DSM': 'Digital and Social Media',
	'DG': 'Digital Government',
	'ES': 'ES',
	'HC': 'Information Technology in Healthcare',
	'IN': 'Internet and the Digital Economy',
	'KS': 'Knowledge Innovation and Entrepreneurial Systems',
	'LI': 'Location Intelligence',
	'OS': 'Organizational Systems and Technology',
	'ST': 'Software Technology',
}

#### PDF extraction utility

In [ ]:
!pip3 install pdfplumber

In [ ]:
import pdfplumber

# Test
filepath = f"./files/{track}/{minitrack_pdf}.pdf"
with pdfplumber.open(filepath) as pdf:
	print(pdf.pages)
	first_page = pdf.pages[0]
	print(first_page.extract_text())

In [5]:
def getPdfText(filename: str):
	"""
	Utility function to extract all text from 
	a pdf. This copies ALL text, not just descriptions (OK for now).

	Args:
		filename: str, full filename with path from current directory
	Returns:
		content: str, text content of the pdf 
	"""
	if filename.endswith('.pdf'):
		with pdfplumber.open(filename) as pdf:
			fullText = []
			for page in pdf.pages:
				fullText.append(page.extract_text())
			content = '\n'.join(fullText)
			return content

#### DOC extraction

In [ ]:
!pip3 install python-docx

In [6]:
import docx

In [7]:
def getDocxText(filename: str):
	"""
	Utility function to extract all text from 
	a .docx file. This copies ALL text, not just descriptions (OK for now).

	Args:
		filename: str, full filename with path from current directory
	Returns:
		content: str, text content of the pdf 
	"""
	doc = docx.Document(filename)
	fullText = []
	for para in doc.paragraphs:
		fullText.append(para.text)
	content = '\n'.join(fullText)
	return content


#### Extract content per minitrack

In [181]:
from configparser import ConfigParser

cfg = ConfigParser()
cfg.read('config.ini')
host=cfg['postgres']['host']
dbname=cfg['postgres']['dbname']
port=cfg['postgres']['port']
user=cfg['postgres']['user']
password=cfg['postgres']['password']

In [ ]:
# using psycopg2 to connect to postgres
# requires postgres driver to be installed: brew install postgres
# requires sqlalchemy to be installed: pip3 install sqlalchemy
# requires importlib to be installed
!pip3 install psycopg2

In [ ]:
with psycopg2.connect("host='{}' port={} dbname='{}' user={} password={}".format(host, port, dbname, user, password)) as conn:
  sql = f"""
    SELECT c.shortname as conference, t.code as track, m.id, m.name, m.description
    FROM minitracks m
    JOIN tracks t on t.id = m.track_id
    JOIN conferences c on c.id = t.conference_id
    where c.shortname='{CONFERENCE}'
  """
  db_minitracks = pd.read_sql_query(sql, conn)
db_minitracks.describe()

In [156]:
mtracks = db_minitracks[(db_minitracks['track'] != 'ES') & (db_minitracks['track'] != 'CSEET')]
print(mtracks.shape)
print(mtracks.describe())

(182, 5)
       conference track                                    id  \
count         182   182                                   182   
unique          1    10                                   182   
top      HICSS-55    DA  a8a72967-8e7e-49ef-a254-62f83f4fe368   
freq          182    34                                     1   

                             name description  
count                         182           0  
unique                        182           0  
top     Digital Government and AI         NaN  
freq                            1         NaN  


In [142]:
print(os.getcwd())
def getFile(name: str, trackcode: str):
	# try searching by first characters first due to some strange issues
	docs = [doc for doc in glob.iglob(f"./{CONFERENCE}/files/{tracksdict[trackcode]}/{name[0:5]}*")]
	if len(docs) == 1: return docs[0]
	if len(docs) > 1:
		# full name search
		print('full name')
		docs = [doc for doc in glob.iglob(f"./{CONFERENCE}/files/{tracksdict[trackcode]}/{name}*")]
		if len(docs) == 1: return docs[0]
	return None

/Users/holmsmidt/hicss/workspace/track-recommender/minitrack-lists


In [159]:
nofilecounter = 0
for index, mt in mtracks.iterrows():
	# quick hack to deal with '/' characters in minitrack names by replacing with an unlikely character '^'
	#print(mt['name'], '  -  ', mt['track'])
	file = getFile(formatMinitrackName(mt['name']), mt['track'])
	description = None
	if file == None:
		print('no file for', mt['track'], ' - ', mt['name'])
		nofilecounter += 1
	elif file.endswith('.docx'): description = getDocxText(file)
	elif file.endswith('.pdf'): description = getPdfText(file)
	if description != None: mtracks.at[index,'description'] = description
print('no of minitracks without file', nofilecounter)

no file for KS  -  Topics in Knowledge, Innovation, and Entrepreneurship Systems
no of minitracks without file 1


In [161]:
# store temp results in csv
mtracks.to_csv(f'./{CONFERENCE}/accepted_minitracks_with_description.csv', sep=',')

In [163]:
mtracks[mtracks.track == 'DA'].head()

,conference,track,id,name,description
38,HICSS-55,DA,3f2e530f-2943-4d51-893f-c38476a4154f,Machine Learning and Predictive Analytics in A...,HICSS’22 minitrack:\nMachine learning and pred...
39,HICSS-55,DA,be080d6a-85f9-45cb-9537-f9a330238639,"Accountability, Evaluation, and Obscurity of A...",HICSS 54: TRACK: Decision Analytics and Servic...
40,HICSS-55,DA,91dd6dfd-c78f-4c36-ac67-291a11b0b348,Algorithm-based Advisory in the Service Sector,Title of a minitrack: \nAlgorithm-based Adviso...
41,HICSS-55,DA,391db4d7-ba7f-432d-a1f4-53805a190e41,Big Data and Analytics: Pathways to Maturity,Minitrack Proposal\nHICSS-55 Decision Analytic...
42,HICSS-55,DA,002583a4-cbd3-4632-905d-d94f0a4a3f3d,Delivering IoT Services using Blockchain,Mini Track Title: Delivering IoT Services usin...


### Update database with minitrack descriptions for HICSS-56

In [173]:
with psycopg2.connect("host='{}' port={} dbname='{}' user={} password={}".format(host, port, dbname, user, password)) as conn:
	cur = conn.cursor()
	for idx, mt in mtracks.iterrows():
		if pd.notna(mt.description):
			try:
				sql = "UPDATE minitracks SET description=%s WHERE id=%s"
				cur.execute(sql, (mt.description, mt.id))
			except:
				print('failed to access description', mt.description)
	conn.commit()